In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'ece697ls/assignments/assignment3/'
FOLDERNAME = None
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content

# Pruning

After learning, neural networks have modified and learned a set of parameters to perform our classification task. However, such parameters are costly to maintain and do not hold the same importance.

Wouldn't it be great could optimize our resource usage by dropping less important values ? This is where pruning comes into play.

Pruning is a technique that cuts off parameters/structures from a model to increase sparcity and decrease overall model size, similar to cutting leafs or branches from bushes and trees. This process can lead to smaller memory consumption with minimal accuracy reduction. Moreover, pruning the network may also provide a speedup since there will be less operations being performed.

The pruning process can be performed during the end of an epoch of training or after training is complete. Experimenting to find out which way works the best is part of the fun !

In [7]:
from ece662.pruning_helper import test_model, load_model
from ece662.data_utils import get_CINIC10_data
import os

Below we will load a pre-trained model for you to work on. If you prefer, you can save your own model from the previous Tensorflow/Pytorch task and load it here.

In [8]:
#This code may take a while to execute as it is training a network form scratch

data = get_CINIC10_data()
mode = 'torch'#torch or tensorflow

test_data = [data['X_test'],data['y_test']]

path = f"ece662/models/{mode}.model"

model = load_model(path,mode=mode)
test_model(model,test_data,mode=mode)

Test Acc: 0.5081


## Unstructured Pruning

Unstructured Pruning is usually related to the pruning of weights in neural networks. The general idea is to select a set of weights according to a policy and setting them up to zero. 

Common policies are random weight selection or selecting the smallers weights. 
Unstructured Pruning can be performed in one or multiple layers within the same network.

Altough in theory Unstructured Pruning should decrease the number of operations performed during execution there should be explicit support within the framework or hardware to bypass such operations, otherwise it will just operated over zero.

### Perform Pruning

Using the model trained in the previous step using pytorch, perform unstructured pruning in the weights of the model by removing x% of the smallest weights. 

*   Increment global pruning by 10% until reaching total of 80% pruned weights
*   Perform inference at the end of each pruning and observe the impact into the accuracy.


Note: The percentages are related to the entire model, not per layer.



In [12]:
################################################################################
# TODO: Perform unstructured Pruning over the trained model using 3 different  
# prunning percentages.                                
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

import torch
import torch.nn.utils.prune as prune
import copy

pruning_percentages = [0.1, 0.2, 0.3]  

for prune_percent in pruning_percentages:
    print(f"Testing {prune_percent*100:.0f}% pruning:")
    
    pruned_model = copy.deepcopy(model)
    
    # Get all parameters of model
    parameters_to_prune = []
    for name, module in pruned_model.named_modules():
        if isinstance(module, (torch.nn.Linear, torch.nn.Conv2d)):
            parameters_to_prune.append((module, 'weight'))
    
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=prune_percent,
    )
    
    # Make pruning permanent
    for module, param in parameters_to_prune:
        prune.remove(module, param)
    
    # Test the pruned model
    test_model(pruned_model, test_data, mode=mode)
    print()

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

Testing 10% pruning:
Test Acc: 0.5091

Testing 20% pruning:
Test Acc: 0.5052

Testing 30% pruning:
Test Acc: 0.5026



## Inline Question 1:

What happened with the accuracy as the % of pruning increased ?
Why was that the case?


## Answer: 

As the percentage of pruning increased, the accuracy of the model decreased. This happens because we are removing weights from the model, across all layers, thus removing learned pattern from the model. 
This will directly decrease the accuracy of the model. For small percentage like 10%, the effect is minimal but it have compound effect as we increase and prune across all layers at 30%. 

## Structured Pruning

Structured Pruning consists of removing a bigger chunk of the network parameters at the same time. Instead of removing only a few weights, it is commonplace to remove entire neurons. 

For example, in Convolutional Layers, removing filters can be beneficial to improve performance as it greatly decreases the amount of computation performed. However, some of these changes may affect output dimensions which may be carried over to other parts of the network. Therefore, when performing structured pruning one must always be aware of which parameters are going to be affected.

Using the previously trained model in the CINIC-10, perform Structured Prunning only in the Convolution layers of the DNN.

In [13]:
################################################################################
# TODO: Perform unstrucuted Pruning over the trained model using 3 different  
# prunning percentages.                                
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Define pruning percentages for structured pruning
structured_pruning_percentages = [0.1, 0.2, 0.3]  # 10%, 20%, 30%

for prune_percent in structured_pruning_percentages:
    print(f"Testing {prune_percent*100:.0f}% structured pruning on conv layers:")
    
    pruned_model = copy.deepcopy(model)
    
    # Apply structured pruning only to convolutional layers
    conv_parameters_to_prune = []
    for name, module in pruned_model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            conv_parameters_to_prune.append((module, 'weight'))
    
    for module, param in conv_parameters_to_prune:
        prune.ln_structured(
            module, 
            name=param, 
            amount=prune_percent, 
            n=1,  
            dim=0  
        )
    
    for module, param in conv_parameters_to_prune:
        prune.remove(module, param)
    
    # Test the pruned model
    test_model(pruned_model, test_data, mode=mode)
    print()

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

Testing 10% structured pruning on conv layers:
Test Acc: 0.3797

Testing 20% structured pruning on conv layers:
Test Acc: 0.2745

Testing 30% structured pruning on conv layers:
Test Acc: 0.1773



## Inline Question 2:

What is the difference between performing Structured Pruning vs Dropout ? 
Why would it be beneficial to perform both techniques when developing a Neural Network?


## Answer: 
I think the key difference between dropout and pruning is that permanence of the removal. For dropout, we temporary drop the neurons during training the help with generalization; but prune is permanent dropping from the model. Thus, dropout would be perfect for regularization, but pruning will definitely hurt accuracy as it lose capacity in exchange for faster performance during inference.